# 以 Weaviate 實作 Naive RAG

參考 url: https://weaviate.io/developers/weaviate/starter-guides/generative

實作文件: https://tender.flybooking.io/wp-content/uploads/2022/10/%E6%94%BF%E5%BA%9C%E6%A8%99%E6%A1%88%E6%80%A5%E8%A8%BA%E5%AE%A4_%E5%9F%BA%E7%A4%8E%E5%85%AC%E6%96%87%E8%AA%B2%E7%A8%8B_20220831.pdf

In [3]:

COHERE_API_KEY = '<自己的api_key>' # 做 Retrieval 的時候會用到
OPENAI_API_KEY = '<自己的api_key>' # 用以調用 language model
HF_TOKEN = '<自己的api_key>' # 可用可不用，端看自己調用的模型需不需要
WEAVIATE_TEST_API_KEY = '<自己的api_key>' # 要去 Weaviate 官網先設置一個 vector_database 然後再去看該 vector_database 裏面的 api_key
WEAVIATE_ENDPOINT = '<和上面的api_key 相對應的 endpoint 網址>' # 和上面的api_key 相對應的 endpoint 網址




In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("政府標案急診室_基礎公文課程_20220831.pdf")
pages = loader.load_and_split()

In [4]:
# # For using WCS
import weaviate
import json
import os
from weaviate.classes.init import AdditionalConfig, Timeout


client = weaviate.connect_to_wcs(
    cluster_url=(WEAVIATE_ENDPOINT),   # 換成自己的 url
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_TEST_API_KEY),  # 使用自己的 API
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-Huggingface-Api-Key": HF_TOKEN,
        "X-Cohere-Api-Key": COHERE_API_KEY,
    },
    additional_config=AdditionalConfig(
        timeout=Timeout(init=2, query=45, insert=120)  # Values in seconds
    )
)

client.is_ready()

True

### Load data

In [7]:
import weaviate
import weaviate.classes as wvc
import os

newcollections = client.collections.create(
    name="MyExampleIndex",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),    # Set the vectorizer to "text2vec-openai" to use the OpenAI API for vector-related operations
    generative_config=wvc.config.Configure.Generative.cohere(),             # Set the generative module to "generative-cohere" to use the Cohere API for RAG
    properties=[
        wvc.config.Property(
            name="content",
            data_type=wvc.config.DataType.TEXT,
            vectorize_property_name=True,  # Include the property name ("question") when vectorizing
            tokenization=wvc.config.Tokenization.LOWERCASE  # Use "lowecase" tokenization
        ),
    ]
)


collection = client.collections.get("MyExampleIndex") # collections.get 需要填入 name 這個參數


with collection.batch.dynamic() as batch:
    for item in pages:
      properties = {
          "content": item.page_content
      }

      batch.add_object( #‵batch.add_data_object`換成‵batch.add_object`
          properties=properties # 沒有‵class_name` 這個參數
      )



In [5]:
collection_name = "MyExampleIndex"

collection = client.collections.get(collection_name) # collections.get 需要填入 name 這個參數

response = collection.generate.near_text(
    query="撰寫公文的原則",
    limit=3,
    grouped_task="以列點的方式來總結關鍵的資訊"
)

response.generated


'以下是從提供的段落中提取的關鍵資訊摘要：\n\n- 寫好公文可以提高效率，並確保各單位瞭解自己的職責。\n- 關鍵是提供清晰、完整的資訊，並確保格式正確和注重細節。\n- 進階公文技巧包括引用相關法規和案例，以及適時展現公司的優勢和成就。\n- 好的公文也可以幫助爭取公司的權益，並帶來成本效益高的收益。\n- 最後，寫好公文可以提高政府機關的好感度，促進合作。'

In [10]:

collection = client.collections.get(collection_name)
response = collection.generate.near_text(
    query="如何撰寫公文",
    limit=3,
    single_prompt="""你是問答任務的助手。
        使用以下檢索到的上下文來回答問題。
        如果你不知道答案，就說你不知道。
        至少使用五個句子並保持答案簡潔。
        根據內文：{content}
        答：
"""
)

In [12]:
for o in response.objects:
    print(o.generated)  # "Single prompt" generations are attributes of each object
    print("原始資料：{}".format(o.properties))
    print("-----------") 

公文寫作是一門藝術，在進階公文寫作時，有幾個關鍵策略可以幫助您爭取公司應有的權益。首先，確保您的公文能引經據典，必要時附上相關機構的函釋，這可以增加您論述的可信度和說服力。其次，適時展現您公司過去的成就和貢獻，這可以讓機構瞭解您公司的能力和價值。如果出現誤判情況，重要的是要為機構留有台階，讓他們有回旋的餘地。好的公文應該言簡意賅，但充滿想像力，讓讀者能感受到您的提議是為了機構的好處。最後，公文寫作成本低、收益高，是爭取權益的有效工具。您提供的範例中提到私校履約爭議，若遇到機關苛刻，除了寫公文外，您也可以嘗試與機關當面協商或尋求上級機構協助，以多管齊下的策略來維護公司權益。
原始資料：{'content': '35政府標案急診室 公文小學堂\n進階公文\n爭取公司應有權益\n公文要能引經據典，必要時應附上公共工程委員會函 釋。\n適當時機可以說明一下自己公司的 豐功偉業。\n如果是其他單位造成機關誤判，要遞台階給機關下。\n好的公文，話不明說，但令人充滿 想像力。\n要展現出「我的提議都是為了你好」\n若寫公文就可以爭取到應有權益 (替公司賺錢 )，絕對是小\n成本大收益！\n進階公文 範例(與私校履約爭議 )\n覺得機關苛刻了，寫了公文\n也沒用，這時該怎麼辦？'}
-----------
公文寫作是一門藝術和科學的結合，可以有效地傳達信息並完成任務。發文單位需要確保公文包含所有相關信息，包括人、事、時、地、物，以便收文單位可以快速理解並採取行動。此外，精準地說明權責分工，並使用「說明、諒達、准予備查」等共通性用語，可以促進不同單位之間的有效溝通和合作。在撰寫公文時，還需要注意減少贅字�贅詞，確保附件寫法正確，並確定正、副本收文單位。這些細節都很重要，可以確保公文傳達清晰、高效，並有助於建立良好的政府形象。
原始資料：{'content': '3政府標案急診室 公文小學堂\n寫公文的要訣\n公文寫得好，可以事半功倍\n發文單位、收文單位各要做甚麼？是依\n照契約哪一條哪一款執行？精準說明權責分工\n詳如說明、諒達 、准予備查、至紉公誼 …\n這些共通性用語可以拉近彼此距離共通性用語\n人、事、時、地、物都要清楚\n要能讓承辦單位收文後速辦資訊充足\n減少贅字贅詞、注意附件寫法 、確定正 副\n本收文單位 …注重細節'}
-----------
公文課程的結束值得慶祝